In [ ]:
# Databricks notebook source
#%pip install -r ../../data_prep_requirements.txt
#dbutils.library.restartPython()

In [ ]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [ ]:
################################################################################### 
# Data Preprocessing Pipeline
#
# This notebook shows an example of a Data Preprocessing pipeline using Unity Catalog.
# It is configured and can be executed as the tasks in the PreprocessRawData workflow defined under
# ``agentops_demo/resources/data-preprocessing-workflow-resource.yml``
#
# Parameters:
# * uc_catalog (required)                     - Name of the Unity Catalog 
# * schema (required)                         - Name of the schema inside Unity Catalog 
# * raw_data_table (required)                 - Name of the raw data table inside UC database
# * preprocessed_data_table (required)        - Name of the preprocessed data table inside UC database
# * hf_tokenizer_model (optional)             - Name of the HuggingFace tokenizer model name
# * max_chunk_size (optional)                 - Maximum chunk size
# * min_chunk_size (optional)                 - Minimum chunk size
# * chunk_overlap (optional)                  - Overlap between chunks 
# * bundle_root (required)                    - Root of the bundle
#
# Widgets:
# * Unity Catalog: Text widget to input the name of the Unity Catalog
# * schema: Text widget to input the name of the database inside the Unity Catalog
# * Raw data table: Text widget to input the name of the raw data table inside the database of the Unity Catalog
# * Preprocessed data table: Text widget to input the name of the preprocessed data table inside the database of the Unity Catalog
# * HuggingFace tokenizer model: Text widget to input the name of the hugging face tokenizer model to import
# * Maximum chunk size: Maximum characters chunks will be split into
# * Minimum chunk size: minimum characters chunks will be split into
# * Chunk overlap: Overlap between chunks
# * Root of bundle: Text widget to input the root of the bundle
#
# Usage:
# 1. Set the appropriate values for the widgets.
# 2. Run the pipeline to chunk the raw data and store in Unity Catalog.
#
##################################################################################

In [ ]:
# List of input args needed to run this notebook as a job.
# Provide them via DB widgets or notebook arguments.

# A Unity Catalog containing the input data
dbutils.widgets.text(
    "uc_catalog",
    "agentops_stacks_dev",
    label="Unity Catalog",
)
# Name of schema
dbutils.widgets.text(
    "schema",
    "agentops",
    label="Schema",
)
# Name of input table
dbutils.widgets.text(
    "raw_data_table",
    "raw_documentation",
    label="Raw data table",
)
# Name of output table
dbutils.widgets.text(
    "preprocessed_data_table",
    "databricks_documentation",
    label="Preprocessed data table",
)
# Name of huggingface tokenizer model
dbutils.widgets.text(
    "hf_tokenizer_model",
    "openai-community/openai-gpt",
    label="HuggingFace tokenizer model",
)
# Maximum chunk size
dbutils.widgets.text("max_chunk_size", "500", label="Maximum chunk size")
# Minimum chunk size
dbutils.widgets.text("min_chunk_size", "20", label="Minimum chunk size")
# Chunk overlap
dbutils.widgets.text("chunk_overlap", "50", label="Chunk overlap")

# Bundle root
dbutils.widgets.text(
    "bundle_root",
    "/",
    label="Root of bundle",
)

### Define input and output variables


In [ ]:
uc_catalog = dbutils.widgets.get("uc_catalog")
schema = dbutils.widgets.get("schema")
raw_data_table = dbutils.widgets.get("raw_data_table")
preprocessed_data_table = dbutils.widgets.get("preprocessed_data_table")
hf_tokenizer_model = dbutils.widgets.get("hf_tokenizer_model")
max_chunk_size = int(dbutils.widgets.get("max_chunk_size"))
min_chunk_size = int(dbutils.widgets.get("min_chunk_size"))
chunk_overlap = int(dbutils.widgets.get("chunk_overlap"))
bundle_root = dbutils.widgets.get("bundle_root")

assert uc_catalog != "", "uc_catalog notebook parameter must be specified"
assert schema != "", "schema notebook parameter must be specified"
assert raw_data_table != "", "raw_data_table notebook parameter must be specified"
assert preprocessed_data_table != "", "preprocessed_data_table notebook parameter must be specified"
assert hf_tokenizer_model != "", "hf_tokenizer_model notebook parameter must be specified"
assert max_chunk_size != "", "max_chunk_size notebook parameter must be specified"
assert min_chunk_size != "", "min_chunk_size notebook parameter must be specified"
assert chunk_overlap != "", "chunk_overlap notebook parameter must be specified"
assert bundle_root != "", "bundle_root notebook parameter must be specified"

# Updating to bundle root
import sys 

root = dbutils.widgets.get("bundle_root")
sys.path.append(root)

### Initialize tokenizer


In [ ]:
# Download tokenizer model to UC volume
from transformers import AutoTokenizer

volume_folder =  f"/Volumes/{uc_catalog}/{schema}/volume_databricks_documentation"

spark.sql(f"CREATE VOLUME IF NOT EXISTS {uc_catalog}.{schema}.volume_databricks_documentation")

# Initialize tokenizer once
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_model, cache_dir=f'{volume_folder}/hg_cache')

### Use the catalog and database specified in the notebook parameters


In [ ]:
spark.sql(f"""USE `{uc_catalog}`.`{schema}`""")

### Create output preprocessed data table


In [ ]:
if not spark.catalog.tableExists(f"{preprocessed_data_table}") or spark.table(f"{preprocessed_data_table}").isEmpty():
  spark.sql(f"""
  CREATE TABLE IF NOT EXISTS {preprocessed_data_table} (
    id BIGINT GENERATED ALWAYS AS IDENTITY,
    url STRING,
    content STRING
  )
  TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true')
  """)

### Create a user-defined function (UDF) to chunk all our documents with spark.


In [ ]:
from functools import partial
import pandas as pd
from pyspark.sql.functions import pandas_udf
from data_preparation.data_preprocessing.preprocessing.create_chunk import split_html_on_p

@pandas_udf("array<string>")
def parse_and_split(
    docs: pd.Series
) -> pd.Series:
    """Parse and split html content into chunks.

    :param docs: Input documents
    :return: List of chunked text for each input document
    """
    
    return docs.apply(lambda html: split_html_on_p(
        html,
        tokenizer=tokenizer,
        chunk_overlap=chunk_overlap,
        min_chunk_size=min_chunk_size,
        max_chunk_size=max_chunk_size
    ))

### Perform data preprocessing.


In [ ]:
from pyspark.sql import functions as F

(spark.table(raw_data_table)
      .filter('text is not null')
      .withColumn('content', F.explode(parse_and_split('text')))
      .drop("text")
      .write.mode('overwrite').saveAsTable(preprocessed_data_table))

In [ ]:
dbutils.notebook.exit(0)